In [1]:
!pip install scikit-learn umap-learn sentence_transformers faiss-cpu plotly matplotlib datasets openai

  Using cached plotly-5.24.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached openai-1.58.1-py3-none-any.whl.metadata (27 kB)
  Using cached pynndescent-0.5.13-py3-none-any.whl.metadata (6.8 kB)
  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached anyio-4.7.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached httpcore-1.0.7-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 1.7 MB/s eta 0:00:00ta 0:00:01
Using cached plotly-5.24.1-py3-none-any.whl (19.1 MB)
Using cached openai-1.58.1-py3-none-any.whl (454 kB)
Using cached anyio-4.7.0-py3-none-any.whl (93 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached ht

In [3]:
from src.text_clustering import ClusterClassifier
from datasets import load_dataset
import os
import json
from glob import glob
from tqdm import tqdm
from openai import OpenAI

In [2]:

texts = []
jsonl_path = "./sample.jsonl"
with open(jsonl_path, "r") as f:
    for line in f:
        texts.append(json.loads(line)["text"])
single_topic_instruction = (
    instruction
) = "위 예시들이 공유하는 공통 주제를 찾아 짧은 문장으로 설명하시오. \
주제의 예시: 경제 및 투자, 건강과 라이프스타일, 아동교육, 환경과 지속가능성, 사회적 정의, 인간권과 평화, 과학기술과 혁신, 문화와 예술, 역사와 문화, 정치와 정부 등. \
또, 그 문서들의 주제의 교육적 가치를 1~10점 사이로 평가하시오. \
어떤 경우에도 절대 나열을 하지 마시오. \
예시의 형식을 반드시 따를것. \
예시1) 주제: 경제 및 투자, 교육적 가치 점수: 8 \n\
예시2) 주제: 상품 광고, 교육적 가치 점수: 2"

cc = ClusterClassifier(embed_model_name="BM-K/KoSimCSE-roberta-multitask", embed_batch_size=512, embed_device="cuda", topic_mode="single_topic", summary_instruction=single_topic_instruction)

# run the pipeline:
embs, labels, summaries = cc.fit(texts)

# show the results
cc.show()

# save  
cc.save(f"./{cc}")

NameError: name 'glob' is not defined

In [19]:
output_jsonl_path = "./output.jsonl"
import json
for i in range(len(cc.texts)):
    label = cc.cluster_labels[[i]][0]
    category = cc.cluster_summaries[label]
    if category is not None and "None" not in category:
        category = category.split(". ")[0].strip()
        with open(output_jsonl_path, "a") as f:
            f.write(json.dumps({"examples": cc.texts[i], "category": category}) + "\n")
